# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [3]:
using Distributed
using MPI

┌ Info: Precompiling MPI [da04e1cc-30fd-572f-bb4f-1f8673147195]
└ @ Base loading.jl:1242


In [4]:
m = MPIManager(np=4)

┌ Warning: MPI cluster manager functionality has been moved to MPIClusterManagers.jl
│   caller = ip:0x0
└ @ Core :-1


MPI.MPIManager(np=4,launched=false,mode=MPI_ON_WORKERS)

In [5]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

4-element Array{Int64,1}:
 2
 3
 4
 5

In [6]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
 @mpi_do m n = rand()

In [7]:
# Let's see if the processors got it
@mpi_do m println(n)

      From worker 4:	0.8234353533667402
      From worker 5:	0.388855048161449
      From worker 3:	0.8953573965387513
      From worker 2:	0.3471409975180517


In [8]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

      From worker 2:	0
      From worker 3:	1
      From worker 5:	3
      From worker 4:	2


In [9]:
# Get the number of processors
@mpi_do m np = MPI.Comm_size(comm)
@mpi_do m println(np)

      From worker 4:	4
      From worker 3:	4
      From worker 2:	4
      From worker 5:	4


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [10]:
using Interact

┌ Info: Recompiling stale cache file /Users/andreasnoack/.julia/compiled/v1.2/Interact/XmYW4.ji for Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1240


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16701755518980868488\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3437262506120203075\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16701755518980868488\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [11]:
@time @mpi_do m mypi = let
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
    mypi
end

      From worker 2:	1.1146639167236572e-13
  1.737365 seconds (683.68 k allocations: 34.518 MiB, 0.48% gc time)


In [10]:
[( @fetchfrom i π-4*mypi, π  ) for i in workers()] 

4-element Array{Tuple{Float64,Irrational{:π}},1}:
 (-5.99999570027876e-8, π = 3.1415926535897...)  
 (-2.0000185063651088e-8, π = 3.1415926535897...)
 (1.9999810252357975e-8, π = 3.1415926535897...) 
 (5.999988594851402e-8, π = 3.1415926535897...)  

In [12]:
function f_serial()
    n = 50_000_000
    h = 1/n
    our_π = 0.0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

function f_serial2(n)
    our_π = 0.0
    for i = 1:n
        x = (i - 0.5)/n
        our_π += 4/(1 + x^2)
    end
    our_π/n
end

f_serial2 (generic function with 1 method)

In [13]:
f_serial() #warmup
f_serial()
f_serial2(50_000_000) #warmup
@time f_serial2(50_000_000)

  0.101282 seconds (5 allocations: 176 bytes)


3.1415926535895617

In [14]:
function f_par(n)

 @mpi_do m begin
    comm = MPI.COMM_WORLD
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        global s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f_par (generic function with 1 method)

In [15]:
@mpi_do m function _pi_sum_par(n)
    comm = MPI.COMM_WORLD

    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    return our_π
end
function f_par2(n)
    @mpi_do m tmp = _pi_sum_par($n)
    @fetchfrom 2 tmp
end
f_par(50_000_000) #warmup
f_par(50_000_000)
f_par2(50_000_000) #warmup
@time f_par2(50_000_000)

  0.027022 seconds (502 allocations: 20.828 KiB)


3.1415926535899046

In [16]:
π

π = 3.1415926535897...

In [17]:
[f_par2(10^k) for k=3:9] .- π

7-element Array{Float64,1}:
  8.333333312293689e-8  
  8.333307377483834e-10 
  8.323119971009874e-12 
  1.1013412404281553e-13
 -1.0702549957386509e-13
  4.2366110619695974e-13
 -2.531308496145357e-14 

In [18]:
@mpi_do m using Elemental
@mpi_do m using LinearAlgebra
@mpi_do m A = Elemental.DistMatrix(Float64)
@mpi_do m Elemental.gaussian!(A, 1000, 800)

In [19]:
@mpi_do m U, s, V = svd(A)
@mpi_do m println(s)
@mpi_do m println(size(U))

      From worker 2:	[59.63999042081751; 59.206168429693356; 59.09271210995488; 58.70169881318562; 58.42339523924266; 57.985960870125325; 57.823127095921144; 57.587270297433975; 57.35118350248538; 57.24088155785849; 56.937517687307185; 56.80419324898116; 56.769201773970494; 56.44044615076583; 56.413147210523476; 56.25877596775589; 56.15793841003025; 55.941702800805366; 55.75922934746711; 55.73252496756081; 55.5326361828278; 55.479483126352655; 55.32101902704133; 55.241302629214424; 55.02688539465607; 54.93329007725278; 54.73580480118247; 54.68001072106168; 54.613542393078085; 54.35442666042533; 54.24508494977557; 54.21356161724148; 54.04010526906814; 53.87881681503667; 53.74691268685868; 53.62285206647892; 53.44065889647314; 53.413110864664425; 53.31756348086409; 53.22745168781336; 53.14103230963622; 52.87888205289889; 52.862467085680926; 52.76829423760861; 52.63655125578984; 52.41002182555969; 52.24837443987195; 52.13535397995886; 52.070554508269105; 51.95871790276112; 51.827646212097